# FLAVA

## 0. Giới thiệu

- Tên paper: __FLAVA: A Foundational Language And Vision Alignment Model__
- Conference: CVPR 2022
- Author: Facebook AI Research (FAIR)
- Link: https://flava-model.github.io/
- FLAVA là một backbone phổ biến, được re-implement bởi các framework: Huggingface Transformers, TorchMultimodal

## 1.  Điểm mới (Main contribution)
- Tạo ra một kiểu pretraining mới: kết hợp __Unimodal Pretraining__ và __Cross-modal Pretraining__ 
    - Ý nghĩa:
        - Tận dụng được nguồn dataset hỗn tạp: image - text pair, image + label, text + label
        - Tạo ra một backbone có thể transfer cho các *cross-modal task* như Visual Question Answering, Image Captioning, Text-to-Image Retrieval,... và các *unimodal task* như Image Classification, Text Classification, ...
    - Tóm tắt sơ lược: phase 1 Unimodal Pretraining; phase2  kết hợp Unimodal và Cross-modal Pretraining
        - After unimodal pretraining of the image and text encoders, we continue training the entire FLAVA model jointly on the three types of datasets with round-robin sampling.
        - In each training iteration, we choose one of the datasets according to a sampling ratio that we determine empirically and obtain a batch of samples.
        - Then, depending on the dataset type, we apply unimodal MIM on image data, unimodal MLM on text data, or the multimodal losses (contrastive, MMM, and ITM).


<!-- image path: imgs/FLAVA_1.png -->
![FLAVA_1](imgs/FLAVA_1.png)

- Đề xuất __Masked Multimodal Modeling (MMM) loss__, tóm tắt lại như sau: 
    - Từ một ảnh thông qua bộ image encoder được mã hóa thành sequence các vector embedding $S_I$
    - Từ một đoạn text thông qua bộ text encoder được mã hóa thành sequence các vector embedding $S_T$
    - Cho concat 2 sequence $S_I$ và $S_T$ thành một sequence $S_{M}$ (__multimodal sequence__). Sau đó cho che một số phần tử của $S_{M}$ thành token [MASK] (tương tự như *BERT* (với text) và *BEiT* (với image)).
    - Masked sequence này sẽ được encode bởi multimodal encoder (còn gọi là cross encoder, fusion encoder). Phần vector embedding output ứng với các token [MASK] sẽ được feed qua một module để dự đoán lại các token bị che.
    - Với các masked text token, cần dự đoán token ứng với từ nào trong bộ word dictionary, còn với các image token bị che là token nào trong bộ image codebook. (Chi tiết task che hình đi đoán lại và khái niệm codebook sẽ được phân tích trong bài viết về BEiT).

![FLAVA_2](imgs/FLAVA_0.png)

## 2. Phân tích mô hình

### 2.1 Kiến trúc

Kiến trúc kinh điển của một Vision-Language model bao gồm 3 thành phần chính: image encoder, text encoder, và multimodal encoder. FLAVA cũng không phải ngoại lệ.

__Image Encoder__

- Kiến trúc Vision Transformer (ViT), cụ thể là ViT-B/16.
- Phía sau Image Encoder là một lớp linear projection để điều chỉnh số chiều của vector embedding.
- Vector embedding ứng với __[CLS]__ token được dùng làm global representation của ảnh. Biểu diễn của token này được dùng để tính loss contrastive. -> mô típ quen thuộc

__Text Encoder__

- Dùng kiến trúc ViT-B/16 giống như Image Encoder
    - Câu hỏi: ViT khác BERT chỗ nào? 
        - Các vision backbone dựa trên transformer thường để lớp Layer Normalization ngay trước module Self-attention
        - Còn BERT để lớp Layer Normalization ngay sau module Self-attention.
    - Image Encoder giống Text Encoder về mặt kiến trúc, chỉ khác nhau ở module embedding: khâu mã hóa input thành sequence của các vector embedding. Nói chung, Text Encoder kiến trúc là ViT-B/16, nhưng module *text embedding* lấy từ BERT.
    - Việc dùng kiến trúc ViT-B/16 tạo ra sự đồng nhất về mặt kiến trúc giữa Image Encoder và Text Encoder.

- Vector embedding ứng với __[CLS]__ token được dùng làm global representation của toàn đoạn text. Biểu diễn của token này được dùng để tính loss contrastive. 

__Multimodal Encoder__

- Kiến trúc là một khối Transformer Encoder. Paper Flava gọi khối này là multimodal encoder, một số paper khác gọi là cross encoder hoặc fusion encoder.

- Multimodal Encoder chỉ dùng self-attention mà không dùng cơ chế cross-attention như bộ Cross Encoder của APTM, ALBEF. Yếu tố 'fusion' trong multimodal encoder của Flava được thể hiện thông qua việc dùng multimodal sequence $S_M$ (concat của $S_I$ và $S_T$) để làm input đầu vào.

- Chú ý:
    - Một ảnh $I$ thông qua image encoder được mã hóa thành một sequence các vector embedding, tạm gọi là $E_I$. Sau đó $E_I$ cần đi qua bộ vision projection để điều chỉnh số chiều, từ đó thu được sequence mới là $S_I$.
    - Tương tự với đoạn text $T$, cần thông qua text encoder và bộ text projection để thu được sequence $S_T$.

- Trong bài báo tóm tắt về Vision-Language model của Microsoft Research, kiểu kiến trúc này được gọi là __merge-attention__.

<!-- Co-attetnion_Merge-attetion.png -->
![Co-attetnion_Merge-attetion](imgs/Co-attetnion_Merge-attetion.png)

### 2.2 Unimodal Pretraining Objectives

Quá trình pretraining của FLAVA sẽ qua hai phase: phase 1 là Unimodal Pretraining, phase 2 là Cross-modal Pretraining. Phần này sẽ phân tích về phase 1.
- Tổng quan phase 1
    - Train text encoder trên tập dữ liệu chỉ có text, train image encoder trên tập dữ liệu chỉ có image.
    - Dùng phương pháp self-supervised, không cần label. 
    - Pretext task __Mask Language Modeling__ (ý tưởng lấy từ BERT), được dùng làm objective để pretrain cho text encoder.
        - We apply a masked language modeling loss on top of the text encoder to pretrain on stand-alone text datasets. A fraction (15%) of the text tokens are masked in the input, and reconstructed from the other tokens using a classifier over the unimodal text hidden states output.
    - Pretext task __Mask Image Modeling__ (ý tưởng lấy từ BEiT), được dùng làm objective để pretrain cho image encoder.
        - On unimodal image datasets, we mask a set of image patches following the rectangular block-wise masking in BEiT and reconstruct them from other image patches. The input image is first tokenized using a pretrained **dVAE tokenizer**. Then a classifier is applied on the image encoder outputs to predict the dVAE tokens of the masked patches.
        - Ý tưởng này có thể tóm lại như sau: Trước tiên cần xây dựng một bộ từ điển cho ảnh gọi là visual codebook (kích thước cố định) và một bộ tokenizer. Với một ảnh kích thước bất kì, đều có thể tokenize thành các visual token rời rạc trong bộ từ điển. Các token trong bộ từ điển là một số điểm trong latent space, mỗi điểm được biểu diễn bằng 1 vector.
        - Câu hỏi đặt ra?
            - Phương pháp tạo ra bộ từ điển và bộ tokenizer?
            - Cơ chế mask image của BEiT?


- Nguồn dataset cho unimodal pretraining:
     - Text dataset: CCNews và BookCorpus
     - Image dataset: ImageNet

- Một điểm đáng chú ý: đối với Image Pretraining, phương pháp self-supervised __DINO__ được các tác giả của FLAVA chỉ ra có performance tốt hơn so với phương pháp dùng Mask Image Modeling như các backbone MAE, BEiT. Nhưng do tính dễ mở rộng và tính đơn giản, MIM task được dùng kết hợp trong quá trình Cross-modal Pretraining ở phase 2.

__Review lại về BEiT__

Masked Language Modeling là một pretext task kinh điển, không có vấn đề để bàn. Do đó, phần này chỉ tập trung review lại BEiT.

![BEiT](imgs/BEiT.png)

- Overview of BEIT pre-training. Before pre-training, we learn an “image tokenizer” via
autoencoding-style reconstruction, where an image is tokenized into __discrete visual tokens__ according
to the __learned vocabulary__. During pre-training, each image has two views, i.e., image patches, and visual tokens. We randomly mask some proportion of image patches (gray patches in the figure) and replace them with a special mask embedding. Then the patches are fed to a backbone vision
Transformer. The pre-training task aims at predicting the visual tokens of the original image based
on the encoding vectors of the corrupted image.

##